In [26]:
%reset -f

In [33]:
import numpy as np # The NumPy library
from scipy.integrate import quad # Method for integration in scipy.integrate sub-package

cells_num = 256

# The sizes are for UQs for number of cells: 4, 8, 16, 32, 64, 128, 256.

# The opitimized spaces for the Guassian distribution in a uniform quantizer(UQ).
all_size_gaus = np.array([0.9957, 0.5860, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308])

'''The opitimized spaces for a Mixed Gaussian distribution in 
a uniform quantizer(UQ). This distribution is not symmetric and the values are
for for cells number 2, 4, 8, 16, 32, 64, 128, 256'''
all_boudary1s = [-0.05, -2.02, -3.17, -3.93, -4.49, -5.08, -5.16, -5.50]
all_deltas = [4, 1.81, 1, 0.54, 0.29, 0.16, 0.08, 0.05]


cell_size = all_size_gaus[int(np.log2(cells_num))-2]


# Boundaries for the symmetric distributions are computed.
boundaries_symm = np.linspace(-(cells_num/2-1) * cell_size, # For symmetric distributions
                               (cells_num/2-1) * cell_size,
                              cells_num-1)

# Boundaries for the asymmetric distributions are computed.
boundary1 = all_boudary1s[int(np.log2(cells_num))-1]
delta = all_deltas[int(np.log2(cells_num))-1]

boundaries_asymm = []
for i in range (0,cells_num-1):
        boundary = boundary1 + (i) * delta
        boundaries_asymm = np.append(boundaries_asymm, boundary)
        
'''To change the symmetric pdf to asymetric pdf, two parts of code 
must be changed include the boundaries and pdf'''        

# boundaries = boundaries_symm # For symmetric distributions
boundaries = boundaries_asymm # For asymetric distributions
        

n_inf = float("-inf")
p_inf = float("inf")

boundaries = np.insert(boundaries, 0, n_inf)
boundaries = np.append(boundaries, p_inf)

def pdf(x): # Defining the distribution
#     gaus_std = 1
#     gaus_mean = 0
#     pdf = 1/(gaus_std*np.sqrt(2*np.pi)) * \
#                   np.exp(-0.5*((x-gaus_mean)/gaus_std)**2) # Gaussian pdf
    
    Mean1 = -2
    Mean2 = 2
    stdev1 = 1
    stdev2 = 1
    alpha = 0.7
    pdf = (alpha)*(1/np.sqrt(2*np.pi*stdev1)*np.exp(-(x-Mean1)**2/(2*stdev1**2)))\
        + (1-alpha)*(1/np.sqrt(2*np.pi*stdev2)*np.exp(-(x-Mean2)**2/(2*stdev2**2)))
    
    return pdf

def xpdf(x):
    xpdf = x * pdf(x)
    return xpdf

def x2pdf (x):
    x2pdf = x * xpdf(x)
    return x2pdf


dijs_s1 = []
dijs_s2 = []
dijs_s3 = []
for i in range (0, int(cells_num/2)):
    prb1, integ_err = quad(pdf, boundaries[i], boundaries[i+1])
    prb2, integ_err = quad(pdf, boundaries[cells_num-(i+1)], boundaries[cells_num-i])
    
    xprb1, integ_err = quad(xpdf, boundaries[i], boundaries[i+1])
    xprb2, integ_err = quad(xpdf, boundaries[cells_num-(i+1)], boundaries[cells_num-i])
    
    x2prb1, integ_err = quad(x2pdf, boundaries[i], boundaries[i+1])
    x2prb2, integ_err = quad(x2pdf, boundaries[cells_num-(i+1)], boundaries[cells_num-i])
    
    yij_s1 = (xprb1+xprb2)/(prb1+prb2)
    
    dij_s1 = x2prb1 + yij_s1**2 * prb1 - yij_s1 *2 * xprb1\
           + x2prb2 + yij_s1**2 * prb2 - yij_s1 *2 * xprb2
    dijs_s1 = np.append(dijs_s1, dij_s1)
    
    
    y1 =  xprb1/prb1
    y2 =  xprb2/prb2
    
    yij_s2 = (y1+y2)/2
    
    dij_s2 = x2prb1 + yij_s2**2 * prb1 - yij_s2 *2 * xprb1\
           + x2prb2 + yij_s2**2 * prb2 - yij_s2 *2 * xprb2
    dijs_s2 = np.append(dijs_s2, dij_s2)
    
    
    
    dij_s3 = 0.5* (x2prb1 + y1**2 * prb1 - y1 *2 * xprb1\
                 + x2prb2 + y1**2 * prb2 - y1 *2 * xprb2\
                 + x2prb1 + y2**2 * prb1 - y2 *2 * xprb1\
                 + x2prb2 + y2**2 * prb2 - y2 *2 * xprb2)
    dijs_s3 = np.append(dijs_s3, dij_s3)

    
D_s1 = sum(dijs_s1)
D_s2 = sum(dijs_s2)
D_s3 = sum(dijs_s3)

D_s1_dB = np.round(10 * np.log10(D_s1),4)
D_s2_dB = np.round(10 * np.log10(D_s2),4)
D_s3_dB = np.round(10 * np.log10(D_s3),4)

print( 'Folded Binary D_s1_dB: ', D_s1_dB)
print( 'Folded Binary D_s2_dB: ', D_s2_dB)
print( 'Folded Binary D_s3_dB: ', D_s3_dB)



Folded Binary D_s1_dB:  2.9189
Folded Binary D_s2_dB:  8.5015
Folded Binary D_s3_dB:  11.5117
